In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
from torch import nn 

# Teaching a quadruped to walk

Time to try out the learning algorithms that you just implemented on a more difficult problem. The WalkerEnv implements a quadruped robot kind-of thing, see for yourself. The goal is to move in the $x$ direction as fast and as far as possible.

Your goal is to implement a class `WalkerPolicy` with function `determine_actions()` just like the StochasticPolicy we used earlier to control the pendulum. Below is a template of this class, but feel free to alter it however you want. The only important thing is the `determine_actions()` function!

After you implement it, copy `WalkerPolicy` into a separate file `WalkerPolicy.py` that you will upload to BRUTE together with the (optional) learned weights in a zip file. How the policy is implemented is up to you! You are constrained to only the libraries we used so far though, such as torch, numpy etc..

You will get some free points just for uploading a working policy (irrelevant of the performance). Further 2 points will be awarded for successfully traversing a small distance in the x direction.


# Hints

There is no single easy way of doing this, but here are some suggestions on what you could try to improve your policy:

1. This problem is much more difficult, than balancing a pendulum. It is a good idea to use a bit larger network than for the pendulum policy.

2. You can also try to use a different optimizer, such as Adam and play with the hyperparameters.

3. Using a neural network to compute the normal distribution scale $\sigma$ can lead to too much randomness in the actions (i.e. exploration). You can use a fixed $\sigma$ instead, or replace it with a learnable `torch.Parameter` initialized to some small constant. Make sure, you run it through an exponential, or softplus function to ensure $\sigma$ is positive.

4. The exploration can also be reduced by penalizing the variance of the action distribution in an additional loss term.

5. If you see some undesirable behaviour, you can tweak the reward function to penalize it. Even though the $x$ distance is all we care about, adding extra terms to the reward can help guide the learning process (This is known as reward shaping). Simply define a reward function mapping the state $s_{t+1}$ and action $a_t$ to a scalar reward $r_t$ and put it in the config dictionary under the key `'reward_fcn'`. See the `WalkerEnv` class for the implementation of the default reward.

6. Using the normal distribution on a bounded action space can lead to certain problems caused by action clipping. This can be mitigated by using a different distribution, such as the Beta distribution. See the `torch.distributions.beta` module for more information. (Note that Beta distribution is defined on the interval [0,1] and works better with parameters $\alpha,\beta \geq 1$.)


In [2]:
# If you cannot run with the visualization, you can set this to False
VISUALIZE = True

In [3]:
from environment.WalkerEnv import WalkerEnv
from WalkerPolicy import WalkerPolicy

In [4]:
def walker_reward(state, action):
    """reward function for the walker environment, state is [29] vector, action is [8] vector"""
    pos = state[:15]  # first 15 elements of state vector are generalized coordinates [xyz, quat, joint_angles]
    vel = state[15:]  # last 14 elements of state vector are generalized velocities [xyz_vel, omega, joint_velocities]
    return vel[0]  # return the x velocity as the reward by default

In [5]:
# This is the configuration for the Walker environment
# N is the number of robots controlled in parallel
# vis is a boolean flag to enable visualization
# !! IMPORTANT track is an integer index to enable camera tracking of a particular robot (indexed by the value of the argument), this is useful when evaluating the performance of the policy after training
# reward_fcn is the reward function that the environment will use to calculate the reward
config = {'N': 1, 'vis': VISUALIZE, "track": 0, "reward_fcn": walker_reward}
env = WalkerEnv(config)

Environment ready


In [6]:
T = 1000
obs = env.vector_reset()  # Observation vector is of shape (N, 29)
print(f"Observation shape is {obs.shape}")
x = obs[0, 0]
for i in range(1000):
    a = np.random.randn(1, 8) * 4 - 2  # Random actions with standard deviation of 2 and mean of 0
    obs, reward = env.vector_step(a)
    x = max(x, obs[0, 0])
env.close()
print(f"After {T} steps, the maximum x value reached was {x}")

Observation shape is (1, 29)


2024-12-18 17:28:19.987 Python[52690:6467138] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/yj/_6b0pqvs5xg33v_xy5q2v9_40000gn/T/org.python.python.savedState
2024-12-18 17:28:20.254 Python[52690:6467138] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-18 17:28:20.254 Python[52690:6467138] +[IMKInputSession subclass]: chose IMKInputSession_Modern


After 1000 steps, the maximum x value reached was 1.5139834880828857
